# TUGAS 2
Buatlah model klasifikasi dengan menggunakan SVM berdasarkan data pada Percobaan 5 dengan menggunakan fitur histogram.

In [2]:
# Import Required Libraries
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pandas as pd

In [3]:
# Image directories
train_dir = "data/images/images/training/"
test_dir = "data/images/images/test/"

In [4]:
# Function to compute histogram
def compute_histogram(image_path):
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()  # Normalize histogram
    return hist

In [7]:
def load_dataset(img_dir):
    p = Path(img_dir)
    img_list = []

    for dir in p.iterdir():
        if dir.is_dir():
            label = dir.name
            for file in dir.glob('*.jpg'):
                try:
                    img = mpimg.imread(file)
                    if img is not None:
                        img_list.append((img, label))
                except Exception as e:
                    print(f"Error loading image {file}: {e}")
    
    return img_list

In [8]:
train_img = load_dataset(train_dir)

In [9]:
test_img = load_dataset(test_dir)

In [11]:
def load_dataset(img_dir):
    p = Path(img_dir)
    img_list = []

    for dir in p.iterdir():
        if dir.is_dir():
            label = dir.name
            for file in dir.glob('*.jpg'):
                try:
                    img = mpimg.imread(file)
                    if img is not None:
                        img_list.append((img, label))
                except Exception as e:
                    print(f"Error loading image {file}: {e}")
    
    return img_list

In [5]:
# Load training data
X = []  # Features
y = []  # Labels

# Iterate through training images
for label in train_dir.glob('*'):
    for img_path in label.glob('*.jpg'):  # Assuming images are in JPG format
        hist = compute_histogram(img_path)
        X.append(hist)
        y.append(label.name)  # Use folder name as label

AttributeError: 'str' object has no attribute 'glob'